In [5]:
#SISI SERVER API
# Hanya dijalankan sekali
!pip install fastapi uvicorn nest_asyncio 

   ---------------------------------------- 0.0/94.6 kB ? eta -:--:--
   ------------------------------ --------- 71.7/94.6 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 94.6/94.6 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/63.7 kB ? eta -:--:--
   ---------------------------------------- 63.7/63.7 kB ? eta 0:00:00
   ---------------------------------------- 0.0/71.5 kB ? eta -:--:--
   ---------------------------------------- 71.5/71.5 kB ? eta 0:00:00


In [9]:
#2. Jalankan Fast_API dengan mode loop
import nest_asyncio
nest_asyncio.apply()

In [16]:
#3. Bagian utama dari FAST-API 
from fastapi import FastAPI
import uvicorn
# Buat instance dari FastAPI
app = FastAPI()
# Buat endpoint sederhana (diakses tanpa parameter)
@app.get("/")
def read_root():
    return {"message": "Hello, FastAPI dari planet Jupyter!"}
# Tambah endpoint dengan parameter : /items/1 (atau 2 atau 3)
@app.get("/items/{item_id}")
def read_item(item_id: int, q: str = None):
    if item_id==1:
        q='Jeruk'
    if item_id==2:
        q='Mangga'
    if item_id==3:
        q='Durian'
    return {"item_id": item_id, "q": q}

# Jalankan server FastAPI menggunakan Uvicorn
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [12008]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51131 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [12008]


In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import openpyxl

# Inisialisasi aplikasi FastAPI
app = FastAPI()

# Nama file Excel
EXCEL_FILE = "products.xlsx"

# Model Pydantic untuk validasi input
class Product(BaseModel):
    name: str
    price: float
    description: str = None

# Fungsi untuk membaca data produk dari Excel
def read_products_from_excel():
    workbook = openpyxl.load_workbook(EXCEL_FILE)
    sheet = workbook.active
    products = {}
    for row in sheet.iter_rows(min_row=2, values_only=True):  # Lewati header
        product_id, name, price, description = row
        products[product_id] = {"name": name, "price": price, "description": description}
    return products
# Fungsi untuk menulis data produk ke Excel
def write_products_to_excel(products):
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.append(["ID", "Name", "Price", "Description"])  # Header
    for product_id, product_data in products.items():
        sheet.append([product_id, product_data["name"], product_data["price"], product_data.get("description", "")])
    workbook.save(EXCEL_FILE)

# Inisialisasi database dari Excel
try:
    fake_products_db = read_products_from_excel()
except FileNotFoundError:
    # Jika file Excel tidak ditemukan, buat file baru dengan data awal
    fake_products_db = {
        1: {"name": "Laptop", "price": 15000, "description": "Laptop gaming high-end"},
        2: {"name": "Smartphone", "price": 7000, "description": "Smartphone terbaru dengan kamera bagus"},
    }
    write_products_to_excel(fake_products_db)

# Endpoint untuk menambahkan produk baru
@app.post("/products/")
def create_product(product: Product):
    new_id = max(fake_products_db.keys()) + 1
    fake_products_db[new_id] = product.dict()
    write_products_to_excel(fake_products_db)  # Simpan ke Excel
    return {"id": new_id, "product": product}

# Endpoint untuk menampilkan semua produk
@app.get("/products/")
def get_products():
    return fake_products_db

# Endpoint untuk menampilkan produk berdasarkan ID
@app.get("/products/{product_id}")
def read_product(product_id: int):
    product = fake_products_db.get(product_id)
    if product is None:
        raise HTTPException(status_code=404, detail="Product not found")
    return product
# Endpoint untuk memperbarui produk berdasarkan ID
@app.put("/products/{product_id}")
def update_product(product_id: int, product: Product):
    if product_id not in fake_products_db:
        raise HTTPException(status_code=404, detail="Product not found")
    fake_products_db[product_id] = product.dict()
    write_products_to_excel(fake_products_db)  # Simpan perubahan ke Excel
    return {"id": product_id, "product": product}

# Endpoint untuk menghapus produk berdasarkan ID
@app.delete("/products/{product_id}")
def delete_product(product_id: int):
    if product_id not in fake_products_db:
        raise HTTPException(status_code=404, detail="Product not found")
    del fake_products_db[product_id]
    write_products_to_excel(fake_products_db)  # Simpan perubahan ke Excel
    return {"message": f"Product {product_id} deleted successfully"}

# Jalankan server FastAPI menggunakan Uvicorn
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop